In [1]:
import pandas as pd 

In [2]:
naver_comments = pd.read_csv("../data/KP_naver_comments_monthlyTop4000.csv")
daum_comments = pd.read_csv("../data/KP_daum_comments_monthlyTop800.csv")

In [3]:
naver_comment_UT = naver_comments[['url', 'text']]
daum_comment_UT = daum_comments[['org_url', 'content']]

daum_comment_UT.columns = ['url', 'text']

In [4]:
news_comments = pd.concat([naver_comment_UT, daum_comment_UT])

In [5]:
news_comments

,url,text
0,https://n.news.naver.com/mnews/article/comment...,네이버 페이보다 수수료 낮을까?
1,https://n.news.naver.com/mnews/article/comment...,공부해야할 과목이 또 하나 늘었군.. - IT종사자 -
2,https://n.news.naver.com/mnews/article/comment...,"네이버도 정기결제, 포인트 적립 다 똑같이 되고 먼저 됐는데요."
3,https://n.news.naver.com/mnews/article/comment...,"네이버페이 좋아요.네이버페이체크카드로 교통카드 사용하면 포인트주고, 천원써도 포인..."
4,https://n.news.naver.com/mnews/article/comment...,네이버페이는 편할거라 생각해서 썼는데 포인트도 제한 교통적립 되긴되는건가?카카오페이...
...,...,...
71387,https://v.daum.net/v/20221226050109898,네이버가 짱이지 ㅋㅋ\n\n올개 30만원 모임\n\n얌얌
71388,https://v.daum.net/v/20221229172340151,손절한 주식은???!!!
71389,https://v.daum.net/v/20221229172340151,배당금이나 주라
71390,https://v.daum.net/v/20221229172340151,어제 새벽부터 카카오페이 안된다!아직도 안되는 이유가 뭐냐?


In [6]:
import comments_sentiment as senti

2023-02-21 13:22:48.177579: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


device : cuda:0
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [7]:
news_comments['senti'] = senti.main(news_comments['text'])

BERT dataset creating......
BERT Classifier Start......


100%|██████████| 3219/3219 [03:43<00:00, 14.43it/s]


Predicting......


In [18]:
pos_comm = news_comments[news_comments['senti'] == 1]
neg_comm = news_comments[news_comments['senti'] == 0]

In [19]:
from konlpy.tag import Okt

In [20]:
okt = Okt()

In [25]:
def tokenizing(x):
    x = str(x)
    if len(x) <  1000:
        return okt.nouns(x)
    else:
        return ['Too Long']

In [26]:
pos_comm['token'] = pos_comm['text'].apply(lambda x : tokenizing(x))
neg_comm['token'] = neg_comm['text'].apply(lambda x : tokenizing(x))

/tmp/ipykernel_24547/2529415391.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pos_comm['token'] = pos_comm['text'].apply(lambda x : tokenizing(x))
/tmp/ipykernel_24547/2529415391.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  neg_comm['token'] = neg_comm['text'].apply(lambda x : tokenizing(x))


In [28]:
pos_comm.to_csv("../data/positive_comments.csv")
neg_comm.to_csv("../data/negative_comments.csv")

In [2]:
import pandas as pd 

pos_comm = pd.read_csv("../data/positive_comments.csv")
neg_comm = pd.read_csv("../data/negative_comments.csv")

In [34]:
from collections import Counter

In [37]:
pos_counter = Counter()
for item in pos_comm['token']:
    pos_counter.update(item)

In [38]:
neg_counter = Counter()
for item in neg_comm['token']:
    neg_counter.update(item)

In [54]:
pos_df = []
for item in zip(pos_counter.keys(), pos_counter.values()):
    pos_df.append(item)
pos_df = pd.DataFrame(pos_df)

In [55]:
neg_df = []
for item in zip(neg_counter.keys(), neg_counter.values()):
    neg_df.append(item)
neg_df = pd.DataFrame(neg_df)

In [66]:
merge_df = pd.merge(pos_df[pos_df[1] > 10 ], neg_df[neg_df[1] > 10 ], how = 'outer', on = 0)

In [70]:
merge_df['1_y'] = merge_df['1_y'].apply(lambda x : x+1)

In [73]:
merge_df['ratio'] = merge_df['1_x'] / merge_df['1_y']

In [78]:
merge_df = merge_df.sort_values('ratio', ascending=False)

In [84]:
merge_df.head(300).to_csv("../result/KP_pos_neg_ratio.csv")

In [99]:
idx_lst = []
for idx, item in enumerate(pos_comm['token']):
    if '베리칩' in item:
        idx_lst.append(idx)

In [100]:
pos_comm.iloc[idx_lst,:]

,url,text,senti,token
7363,https://n.news.naver.com/mnews/article/comment...,다음은 설마 베리칩? 에이,1,"[다음, 설마, 베리칩, 에이]"
7903,https://n.news.naver.com/mnews/article/comment...,다음은 설마 베리칩? 에이,1,"[다음, 설마, 베리칩, 에이]"
9133,https://n.news.naver.com/mnews/article/comment...,다음은 설마 베리칩? 에이,1,"[다음, 설마, 베리칩, 에이]"
20727,https://n.news.naver.com/mnews/article/comment...,여러분 편하고 좋은것같지만 마냥 좋은것이아닙니다 666표가 대중화되기위한 전 밑밥작...,1,"[여러분, 것, 마냥, 것, 표, 대중화, 위, 전, 밑밥, 작업, 일, 사람, 은..."
20739,https://n.news.naver.com/mnews/article/comment...,현금을 써야 되요... 베리칩 이라고 들어 보셨습니까?,1,"[현금, 베리칩]"
58184,https://n.news.naver.com/mnews/article/comment...,왜? 베리칩 심게 ? 미리 말해두는데 베리칩은 반대다.,1,"[왜, 베리칩, 미리, 말, 베리칩, 반대]"
58208,https://n.news.naver.com/mnews/article/comment...,왜? 베리칩 심게 ? 미리 말해두는데 베리칩은 반대다.,1,"[왜, 베리칩, 미리, 말, 베리칩, 반대]"
59207,https://n.news.naver.com/mnews/article/comment...,유튜브 - 일루미나티 빌게이츠 베리칩 - 그들의 계획 - 모두 보기를..,1,"[유튜브, 일루미나티, 빌게이츠, 베리칩, 그, 계획, 모두, 보기]"
59537,https://n.news.naver.com/mnews/article/comment...,조만간 베리칩 나온다는 소리하겠넹,1,"[조만간, 베리칩, 소리, 넹]"
59595,https://n.news.naver.com/mnews/article/comment...,이러면서 슬슬 베리칩이 등장하지,1,"[슬슬, 베리칩, 등장]"


### 결론

의미 없음

그렇다면 ESG 댓글과의 차이를 볼 수는 있나?

In [6]:
# pos_comm = pd.read_csv("../data/positive_comments.csv")
# neg_comm = pd.read_csv("../data/negative_comments.csv")

In [2]:
import pandas as pd
pos = pd.read_csv("../data/positive_comments.csv")
neg = pd.read_csv("../data/negative_comments.csv")

In [4]:
print(pos.shape[0], neg.shape[0])

71381 340608


In [2]:
import pandas as pd
total, esg = [22444, 389545], [29, 364]
x = pd.DataFrame([total, esg], columns=['pos', 'neg'], index = ['total', 'esg'])
x

,pos,neg
total,22444,389545
esg,29,364


In [3]:
from scipy.stats import chi2_contingency
chi2, p, dof, expected = chi2_contingency([total,esg])

msg = 'Test Statistic: {}\np-value: {}\nDegree of Freedom: {}'
print(msg.format(chi2, p, dof))
print(expected)

Test Statistic: 2.4800460054648914
p-value: 0.11529888024620341
Degree of Freedom: 1
[[2.24515832e+04 3.89537417e+05]
 [2.14167665e+01 3.71583234e+02]]


In [8]:
pos_comm.shape[0] / neg_comm.shape[0]

0.20956935832393836

In [6]:
esg_comments = pd.read_csv("../data/ESG_split/comments_by_esg.csv")

In [8]:
import comments_sentiment as senti

esg_comments['senti'] = senti.main(esg_comments['text'])

2023-02-22 16:37:20.954038: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


device : cuda:0
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/.cache/kobert_v1.zip
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/mglee/VSCODE/git_folder/KakaoPay_ESG/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
BERT dataset creating......
BERT Classifier Start......


100%|██████████| 4/4 [00:01<00:00,  2.78it/s]

Predicting......


In [17]:
sum(esg_comments['senti'] == 1) / sum(esg_comments['senti'] == 0)

0.2204968944099379

이것도 의미 없음...

In [42]:
esg_comments.groupby('esg').sum('senti')

,senti
esg,
ESG추진위원회,5
QR체크인재활용,4
TCFD지지선언,0
UNGC가입,1
같이가치와연말기부캠페인,1
계좌지킴이서비스,1
금융사기및송금피해예방,0
금융취약계층 응대 매뉴얼,0
동네무료보험서비스,0


In [44]:
esg_comments.groupby('esg').size()

esg
ESG추진위원회           11
QR체크인재활용           14
TCFD지지선언            5
UNGC가입              6
같이가치와연말기부캠페인        4
계좌지킴이서비스            3
금융사기및송금피해예방         1
금융취약계층 응대 매뉴얼       1
동네무료보험서비스           1
상생기금 10억 조성         2
아름다운재단 기부           1
악성앱탐지솔류션적용        169
영세중소상공인수수료우대       24
자립준비청년금융안심보험기부    128
지속가능경영보고서약속         1
풍수해보험MOU           22
dtype: int64

: 

In [41]:
esg_comments.groupby('esg').sum('senti')['senti'] /  esg_comments.groupby('esg').size().values

esg
ESG추진위원회          0.454545
QR체크인재활용          0.285714
TCFD지지선언          0.000000
UNGC가입            0.166667
같이가치와연말기부캠페인      0.250000
계좌지킴이서비스          0.333333
금융사기및송금피해예방       0.000000
금융취약계층 응대 매뉴얼     0.000000
동네무료보험서비스         0.000000
상생기금 10억 조성       0.500000
아름다운재단 기부         0.000000
악성앱탐지솔류션적용        0.147929
영세중소상공인수수료우대      0.208333
자립준비청년금융안심보험기부    0.195312
지속가능경영보고서약속       1.000000
풍수해보험MOU          0.090909
Name: senti, dtype: float64